# Retail Sales Analytics Dashboard



In [1]:
# Importing all the important libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set style for professional visualizations
plt.style.use('default')  # Changed from seaborn-v0_8 for compatibility
sns.set_palette("husl")

In [3]:
 # Load data
data = pd.read_csv("FootWare_Wholesale_Sales_Dataset.csv")

In [4]:
# Data cleaning
data.drop_duplicates(inplace=True)
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data = data.dropna()

In [5]:
# Map your columns to expected column names
column_mapping = {
        'Total Revenue (₹)': 'Revenue',
        'Quantity Sold': 'Units Sold',
        'Unit Price (₹)': 'Price',
        'Brand': 'Category',  # Using Brand as Category since you don't have Category
        'Dealer Location': 'Region'  # Using Dealer Location as Region
}

In [6]:
# Rename columns to match expected names
data = data.rename(columns=column_mapping)

In [7]:
# Create a synthetic OrderID based on row index
data['OrderID'] = data.index + 1

In [8]:
 # Create additional columns for analysis
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year
data['Quarter'] = data['Date'].dt.quarter
data['Day_of_Week'] = data['Date'].dt.day_name()
data['Month_Name'] = data['Date'].dt.month_name()

In [9]:
# Clean up Region names (remove extra spaces, standardize)
if 'Region' in data.columns:
  data['Region'] = data['Region'].str.strip().str.title()

In [10]:
 # Clean up Category names
if 'Category' in data.columns:
  data['Category'] = data['Category'].str.strip().str.title()

In [67]:
# Print summary

print("EXECUTIVE SUMMARY")
print("=" * 60)

print(f"\n Dataset Overview:")
print(f"  • Total Records: {len(data):,}")
print(f"  • Date Range: {data['Date'].min().strftime('%Y-%m-%d')} to {data['Date'].max().strftime('%Y-%m-%d')}")
print(f"  • Products: {kpis['total_products']}")
print(f"  • Brands: {kpis['total_categories']}")
print(f"  • Regions: {kpis['total_regions']}")

EXECUTIVE SUMMARY

 Dataset Overview:
  • Total Records: 3,054
  • Date Range: 2023-01-01 to 2024-01-02
  • Products: 5
  • Brands: 10
  • Regions: 4


In [64]:
print(f" Data loaded successfully! Shape: {data.shape}")

 Data loaded successfully! Shape: (3054, 22)


In [65]:
print(f" Available columns: {list(data.columns)}")

 Available columns: ['Date', 'Product', 'Category', 'Size', 'Units Sold', 'Price', 'Margin (%)', 'Profit (₹)', 'Net Profit (₹)', 'Revenue', 'Tax (GST % )', 'Tax Amount (₹)', 'Net Tax (₹)', 'Dealer', 'Stock Availability', 'Region', 'OrderID', 'Month', 'Year', 'Quarter', 'Day_of_Week', 'Month_Name']


In [66]:
data

,Date,Product,Category,Size,Units Sold,Price,Margin (%),Profit (₹),Net Profit (₹),Revenue,...,Net Tax (₹),Dealer,Stock Availability,Region,OrderID,Month,Year,Quarter,Day_of_Week,Month_Name
0,2023-01-01,Sneakers,Nike,10,27,5938.70,15%,890.81,24051.87,136293.03,...,19241.28,Dealer_2,1,Delhi,1,1,2023,1,Sunday,January
1,2023-01-01,Flats,Paragon,9,15,1976.23,40%,790.49,11857.35,17786.10,...,3557.25,Dealer_1,57,Delhi,2,1,2023,1,Sunday,January
2,2023-01-01,Sneakers,Adidas,7,42,6816.15,15%,1022.42,42941.64,243336.66,...,34353.48,Dealer_6,100,Pune,3,1,2023,1,Sunday,January
3,2023-01-01,Flats,Paragon,10,44,1548.80,40%,619.52,27258.88,40888.32,...,8177.84,Dealer_4,200,Haryana,4,1,2023,1,Sunday,January
4,2023-01-01,Sandals,Nike,11,20,11377.67,15%,1706.65,34133.00,193420.40,...,27306.40,Dealer_3,121,Mumbai,5,1,2023,1,Sunday,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3049,2024-01-02,Sandals,Paragon,11,50,1628.68,40%,651.47,32573.50,48860.50,...,9772.00,Dealer_5,128,Haryana,3050,1,2024,1,Tuesday,January
3050,2024-01-02,Flats,Nike,8,38,10081.47,15%,1512.22,57464.36,325631.50,...,45971.64,Dealer_7,139,Mumbai,3051,1,2024,1,Tuesday,January
3051,2024-01-02,Sandals,Reebok,8,21,7674.62,20%,1534.92,32233.32,128933.70,...,19339.95,Dealer_2,172,Delhi,3052,1,2024,1,Tuesday,January
3052,2024-01-02,Heels,Nike,7,26,8540.95,15%,1281.14,33309.64,188755.06,...,26647.66,Dealer_1,78,Delhi,3053,1,2024,1,Tuesday,January


##Advanced KPI calculations

In [12]:
kpis = {}

In [13]:
# Basic KPIs
kpis['total_revenue'] = data['Revenue'].sum()
kpis['total_units'] = data['Units Sold'].sum()
kpis['total_orders'] = len(data)  # Each row is an order in your data
kpis['avg_order_value'] = kpis['total_revenue'] / kpis['total_orders']

In [14]:
# Advanced KPIs
kpis['avg_revenue_per_unit'] = kpis['total_revenue'] / kpis['total_units']
kpis['total_products'] = data['Product'].nunique()
kpis['total_categories'] = data['Category'].nunique()
kpis['total_regions'] = data['Region'].nunique()

In [15]:
# Performance metrics
kpis['best_selling_product'] = data.groupby('Product')['Units Sold'].sum().idxmax()
kpis['highest_revenue_product'] = data.groupby('Product')['Revenue'].sum().idxmax()
kpis['best_region'] = data.groupby('Region')['Revenue'].sum().idxmax()
kpis['best_category'] = data.groupby('Category')['Revenue'].sum().idxmax()

In [16]:
# Growth metrics (comparing first and last month)
monthly_revenue = data.groupby(data['Date'].dt.to_period('M'))['Revenue'].sum()
if len(monthly_revenue) > 1:
 kpis['revenue_growth_rate'] = ((monthly_revenue.iloc[-1] - monthly_revenue.iloc[0]) / monthly_revenue.iloc[0]) * 100
else:
  kpis['revenue_growth_rate'] = 0

In [17]:
# Efficiency metrics
kpis['revenue_per_product'] = kpis['total_revenue'] / kpis['total_products']
kpis['orders_per_day'] = kpis['total_orders'] / (data['Date'].max() - data['Date'].min()).days
kpis

{'total_revenue': np.float64(295679041.03999996),
 'total_units': np.int64(91614),
 'total_orders': 3054,
 'avg_order_value': np.float64(96816.97480026195),
 'avg_revenue_per_unit': np.float64(3227.4438518130414),
 'total_products': 5,
 'total_categories': 10,
 'total_regions': 4,
 'best_selling_product': 'Flats',
 'highest_revenue_product': 'Flats',
 'best_region': 'Delhi',
 'best_category': 'Nike',
 'revenue_growth_rate': np.float64(-92.05287173848672),
 'revenue_per_product': np.float64(59135808.20799999),
 'orders_per_day': 8.344262295081966}

In [19]:
# Display KPIs first
print("KEY PERFORMANCE INDICATORS")
print("="*80)
print(f"TOTAL REVENUE: ₹{kpis['total_revenue']:,.2f}")
print(f"TOTAL UNITS SOLD: {kpis['total_units']:,}")
print(f"TOTAL ORDERS: {kpis['total_orders']:,}")
print(f"AVERAGE ORDER VALUE: ₹{kpis['avg_order_value']:.2f}")
print(f"REVENUE PER UNIT: ₹{kpis['avg_revenue_per_unit']:.2f}")
print(f"BEST SELLING PRODUCT: {kpis['best_selling_product']}")
print(f"TOP REVENUE PRODUCT: {kpis['highest_revenue_product']}")
print(f"BEST PERFORMING REGION: {kpis['best_region']}")
print(f"GROWTH RATE: {kpis['revenue_growth_rate']:.1f}%")
print(f"REVENUE PER PRODUCT: ₹{kpis['revenue_per_product']:,.0f}")
print(f"ORDERS PER DAY: {kpis['orders_per_day']:.1f}")
print("="*80)

KEY PERFORMANCE INDICATORS
TOTAL REVENUE: ₹295,679,041.04
TOTAL UNITS SOLD: 91,614
TOTAL ORDERS: 3,054
AVERAGE ORDER VALUE: ₹96816.97
REVENUE PER UNIT: ₹3227.44
BEST SELLING PRODUCT: Flats
TOP REVENUE PRODUCT: Flats
BEST PERFORMING REGION: Delhi
GROWTH RATE: -92.1%
REVENUE PER PRODUCT: ₹59,135,808
ORDERS PER DAY: 8.3


## Interactive Visualizations

In [25]:
# 1. Monthly Revenue Trend
monthly_data = data.groupby(data['Date'].dt.to_period('M'))['Revenue'].sum().reset_index()
monthly_data['Date'] = monthly_data['Date'].dt.to_timestamp()

fig1 = px.line(monthly_data, x='Date', y='Revenue',
               title='-> Monthly Revenue Trend',
               markers=True)
fig1

In [26]:
#2. Revenue by Region
region_stats = data.groupby('Region').agg({
        'Revenue': 'sum',
        'Units Sold': 'sum',
        'OrderID': 'nunique'
    }).reset_index()
region_stats.rename(columns={'OrderID': 'Orders'}, inplace=True)

fig2 = px.bar(region_stats, x='Region', y='Revenue',
                  title='-> Regional Performance Dashboard',
                  labels={'Revenue': 'Revenue (₹)', 'Region': 'Region'},
                  color='Revenue', color_continuous_scale='viridis',
                  hover_data={'Units Sold': ':,', 'Orders': ':,'})
fig2.update_layout(title_font_size=18, height=500, showlegend=False)
fig2.show()

In [27]:
# 3. Top 10 Products by Revenue
top_products = data.groupby('Product')['Revenue'].sum().nlargest(10).reset_index()

fig3 = px.bar(top_products, x='Revenue', y='Product',
              orientation='h',
              title='-> Top 10 Products by Revenue')
fig3

In [30]:
# 4. Category (Brand) Performance
category_stats = data.groupby('Category').agg({
        'Revenue': 'sum',
        'Units Sold': 'sum',
        'OrderID': 'nunique',
        'Price': 'mean'
    }).reset_index()
category_stats.rename(columns={'OrderID': 'Orders'}, inplace=True)

fig4 = px.pie(category_stats, values='Revenue', names='Category',
                  title='-> Revenue Distribution by Brand',
                  hover_data={'Revenue': ':,.0f', 'Units Sold': ':,', 'Orders': ':,'})
fig4.update_traces(textposition='inside', textinfo='percent+label',
                       hovertemplate='<b>%{label}</b><br>Revenue: ₹%{value:,.0f}<br>Percentage: %{percent}<extra></extra>')
fig4.update_layout(title_font_size=18, height=500)
fig4.show()

In [31]:
 # 5. Top Products Analysis
product_stats = data.groupby('Product').agg({
        'Revenue': 'sum',
        'Units Sold': 'sum',
        'OrderID': 'nunique',
        'Price': 'mean'
    }).reset_index().sort_values('Revenue', ascending=False)
product_stats.rename(columns={'OrderID': 'Orders'}, inplace=True)

# Take top 15 products for better visualization
top_products = product_stats.head(15)

fig5 = px.bar(top_products, x='Product', y='Revenue',
                  title='-> Top Product Performance Ranking',
                  labels={'Revenue': 'Revenue (₹)', 'Product': 'Product'},
                  color='Revenue', color_continuous_scale='plasma',
                  hover_data={'Units Sold': ':,', 'Orders': ':,', 'Price': ':,.0f'})
fig5.update_layout(title_font_size=18, xaxis_tickangle=-45, height=600, showlegend=False)
fig5.show()

In [33]:
# 6. Price vs Units Sold Analysis
fig6 = px.scatter(data, x='Price', y='Units Sold',
                      color='Category', size='Revenue',
                      title='-> Price vs Units Sold Analysis',
                      labels={'Price': 'Price (₹)', 'Units Sold': 'Units Sold'},
                      hover_data={'Product': True, 'Region': True, 'Revenue': ':,.0f'})
fig6.update_layout(title_font_size=18, height=600)
fig6.show()

In [34]:
 # 7. Order Value Distribution
fig7 = px.histogram(data, x='Revenue', nbins=30,
                       title='-> Order Value Distribution',
                       labels={'Revenue': 'Order Value (₹)', 'count': 'Number of Orders'},
                       color_discrete_sequence=['skyblue'])
fig7.add_vline(x=data['Revenue'].mean(), line_dash="dash", line_color="red",
                   annotation_text=f"Mean: ₹{data['Revenue'].mean():.0f}")
fig7.add_vline(x=data['Revenue'].median(), line_dash="dash", line_color="green",
                   annotation_text=f"Median: ₹{data['Revenue'].median():.0f}")
fig7.update_layout(title_font_size=18, height=500)
fig7.show()

In [35]:
 # 8. Regional-Category Heatmap
heatmap_data = data.pivot_table(values='Revenue', index='Region', columns='Category', aggfunc='sum', fill_value=0)
fig8 = px.imshow(heatmap_data.values,
                     x=heatmap_data.columns,
                     y=heatmap_data.index,
                     color_continuous_scale='YlOrRd',
                     title='-> Regional-Brand Performance Heatmap',
                     labels={'color': 'Revenue (₹)'})
fig8.update_layout(title_font_size=18, height=500)
fig8.show()

In [37]:
#9. High-Value Orders Analysis
revenue_threshold = data['Revenue'].quantile(0.8)  # Top 20%
high_value = data[data['Revenue'] > revenue_threshold].copy()

fig9 = px.scatter(high_value, x='Date', y='Revenue',
                     color='Region', size='Units Sold',
                     title=f'-> High-Value Orders Analysis (Top 20% >₹{revenue_threshold:.0f})',
                     hover_data={'Product': True, 'Category': True})
fig9.update_layout(title_font_size=18, height=500)
fig9.show()

In [38]:
# 10. Price Range Analysis by Category
fig10 = px.box(data, x='Category', y='Price',
                  title='-> Price Range Analysis by Brand',
                  labels={'Price': 'Price (₹)', 'Category': 'Brand'},
                  color='Category')
fig10.update_layout(title_font_size=18, height=500, xaxis_tickangle=-45)
fig10.show()

In [39]:
# 11. Profit Analysis (using your Profit column)
if 'Profit (₹)' in data.columns:
  profit_by_product = data.groupby('Product')['Profit (₹)'].sum().sort_values(ascending=False).head(10)

fig11 = px.bar(x=profit_by_product.values, y=profit_by_product.index,
                      orientation='h',
                      title='->  Top 10 Products by Profit',
                      labels={'x': 'Profit (₹)', 'y': 'Product'},
                      color=profit_by_product.values, color_continuous_scale='Greens')
fig11.update_layout(title_font_size=18, height=600)
fig11.show()

##  Extract Key Business Insights

In [58]:
insights = []

# 1. Peak Month
monthly_revenue = data.groupby(data['Date'].dt.month)['Revenue'].sum()
peak_month = monthly_revenue.idxmax()
month_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
               7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
insights.append(f"-> [{month_names[peak_month]}] shows peak performance – ideal for campaigns.")

# 2. Product performance
top_product_revenue = data.groupby('Product')['Revenue'].sum().max()
total_revenue = kpis['total_revenue']
top_product_share = (top_product_revenue / total_revenue) * 100
insights.append(f"-> Top product '{kpis['highest_revenue_product']}' contributes {top_product_share:.1f}% of total revenue.")


# 3. Regional performance
region_performance = data.groupby('Region')['Revenue'].sum()
best_region_share = (region_performance.max() / total_revenue) * 100
insights.append(f"-> '{kpis['best_region']}' region leads with {best_region_share:.1f}% of total revenue.")


# 4. Fastest-Growing Category
category_monthly = data.groupby(['Month', 'Category'])['Revenue'].sum().unstack()
growth_rates = (category_monthly.iloc[-1] - category_monthly.iloc[0]) / category_monthly.iloc[0]
fastest_growth_category = growth_rates.idxmax()
insights.append(f"-> Category showing fastest growth: {fastest_growth_category}.")

# 5. Weekday Performance
weekday_sales = data.groupby('Day_of_Week')['Revenue'].mean().idxmax()
insights.append(f"-> [{weekday_sales}] is the most profitable day on average.")

# 6. Average order value analysis
if kpis['avg_order_value'] > 1000:
    insights.append(f"-> Strong AOV of ₹{kpis['avg_order_value']:.2f} indicates premium customer base.")
else:
    insights.append(f"-> AOV of ₹{kpis['avg_order_value']:.2f} suggests opportunity for upselling.")

# 7. # Category insights
category_count = data.groupby('Category')['Revenue'].sum()
dominant_category = category_count.idxmax()
dominant_share = (category_count.max() / total_revenue) * 100
insights.append(f"-> '{dominant_category}' brand dominates with {dominant_share:.1f}% revenue share.")

# 8. Profit insights (if available)
if 'Profit (₹)' in data.columns:
    total_profit = data['Profit (₹)'].sum()
    profit_margin = (total_profit / total_revenue) * 100
    insights.append(f"-> Overall profit margin is {profit_margin:.1f}%")

# Revenue insights
if kpis['revenue_growth_rate'] > 0:
    insights.append(f"-> Positive growth trend with {kpis['revenue_growth_rate']:.1f}% revenue growth")
else:
    insights.append(f"-> Revenue declined by {abs(kpis['revenue_growth_rate']):.1f}% - 'needs attention'")



# Show Insights
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

1. -> [Aug] shows peak performance – ideal for campaigns.
2. -> Top product 'Flats' contributes 36.1% of total revenue.
3. -> 'Delhi' region leads with 34.7% of total revenue.
4. -> Category showing fastest growth: Woodland.
5. -> [Thursday] is the most profitable day on average.
6. -> Strong AOV of ₹96816.97 indicates premium customer base.
7. -> 'Nike' brand dominates with 37.7% revenue share.
8. -> Overall profit margin is 0.9%
9. -> Revenue declined by 92.1% - 'needs attention'


## Additional Information

In [68]:
# Dealer

if 'Dealer' in data.columns:
  # Dealer performance
  dealer_stats = data.groupby('Dealer').agg({
            'Revenue': 'sum',
            'Units Sold': 'sum',
            'Profit (₹)': 'sum' if 'Profit (₹)' in data.columns else 'count'
        }).reset_index().sort_values('Revenue', ascending=False)

# Top 10 dealers
  top_dealers = dealer_stats.head(10)

  fig = px.bar(top_dealers, x='Revenue', y='Dealer', orientation='h',
                     title='-> Top 10 Dealers by Revenue',
                     labels={'Revenue': 'Revenue (₹)', 'Dealer': 'Dealer'},
                     color='Revenue', color_continuous_scale='Blues')
  fig.update_layout(title_font_size=18, height=600)
  fig.show()
  print(dealer_stats)

else:
  print(" Dealer column not found in data")

     Dealer      Revenue  Units Sold  Profit (₹)
5  Dealer_6  73187573.52       23241   693869.71
1  Dealer_2  67519644.34       20260   618988.75
2  Dealer_3  44645544.36       14410   436692.07
6  Dealer_7  40098092.98       12144   369742.24
0  Dealer_1  35002300.46       10121   316835.09
3  Dealer_4  22167398.46        7311   207236.01
4  Dealer_5  13058486.92        4127   119333.78


In [69]:
# Stock analysis using your Stock Availability column

if 'Stock Availability' in data.columns:
  # Stock vs Sales correlation
  fig = px.scatter(data, x='Stock Availability', y='Units Sold',
                        color='Category', size='Revenue',
                        title='-> Stock Availability vs Sales Performance',
                        labels={'Stock Availability': 'Stock Level', 'Units Sold': 'Units Sold'},
                        hover_data={'Product': True, 'Revenue': ':,.0f'})
  fig.update_layout(title_font_size=18, height=600)
  fig.show()

  # Stock levels by product
  stock_summary = data.groupby('Product').agg({
            'Stock Availability': 'mean',
            'Units Sold': 'sum',
            'Revenue': 'sum'
  }).reset_index().sort_values('Revenue', ascending=False)
  print(stock_summary)
else:
  print("Stock Availability column not found in data")

    Product  Stock Availability  Units Sold       Revenue
1     Flats          123.614312       32403  1.066839e+08
3   Sandals          124.680460       26276  8.204400e+07
4  Sneakers          121.792846       19869  6.421461e+07
2     Heels          127.840708        6666  2.218803e+07
0     Boots          124.421801        6400  2.054853e+07


In [63]:
print(" All analyses completed!")

 All analyses completed!
